In [1]:
'''
Importamos las librerias necesarias para procesar los datos.
'''
import numpy as np
import pandas as pd
import xarray
import netCDF4
import wrf 
import sys
#np.set_printoptions(threshold=sys.maxsize) # Para que las matrices se impriman completas y no resumidas

In [2]:
'''
Parametros
'''
hora1 = "06:00:00"
hora2 = "18:00:00"
dia = "2019-01-01"
hora = hora1
dataDIR = "../Datos/MatricesX/wrfout_A_d01_{}_{}".format(dia,hora)

In [3]:
'''
Abrir el dataset como una matriz XARRAY y mostrar información de sus parametros
'''
DS = xarray.open_dataset(dataDIR)


In [5]:
wrf.getvar(DS, 'z', msl=False) # si no se indica timeidx, selecciona el tiempo 0

TypeError: iteration over a 0-d array

In [159]:
PH_numpy = DS.PH.sel(Time = 6, bottom_top_stag = 10 ).values #shape (269, 269)
PHB_numpy = DS.PHB.sel(Time = 6, bottom_top_stag = 10 ).values #shape (269, 269)
z = (PH_numpy + PHB_numpy) / 9.81 # shape (269, 269)
print z

[[3018.7932 3037.6719 3042.5974 ... 2209.246  2208.648  2208.0723]
 [3145.1294 3158.653  3145.7205 ... 2210.4272 2209.734  2208.5315]
 [3272.2087 3275.4888 3229.9517 ... 2211.1658 2210.2986 2209.0137]
 ...
 [5955.592  6095.3867 6058.616  ... 2331.8062 2332.2239 2332.4749]
 [5989.1167 6297.7227 6263.0713 ... 2332.289  2332.073  2332.365 ]
 [5929.934  6319.695  6263.473  ... 2332.737  2332.562  2332.6804]]


In [78]:
'''
Mostrar informacion de una variable especifica del dataset 
(se lee el dataset como NETCDF y la info de la variable se muestra como una matriz xarray).
'''
#wrf.getvar(ds, 'z', timeidx=wrf.ALL_TIMES, msl=False) # si no se indica timeidx, selecciona el tiempo 0
#wrf.getvar(ds, 'PHB', timeidx=wrf.ALL_TIMES)
#print wrf.getvar(ds, 'z', timeidx=wrf.ALL_TIMES, msl=False, units='km').shape

'\nMostrar informacion de una variable especifica del dataset \n(se lee el dataset como NETCDF y la info de la variable se muestra como una matriz xarray).\n'

In [ ]:
'''
Leer el atributo "Description" de la variable indicada del dataset.
'''
print wrf.getvar(ds, 'PH', timeidx=wrf.ALL_TIMES).description

'''
El sgte. codigo imprime las descripciones de todas las variables: 

a = ds.variables.keys()
for x in a:
    print x, wrf.getvar(ds, x, timeidx=wrf.ALL_TIMES).description
'''
print

In [ ]:
'''
Seleccionar un subset de datos de una variable especifica, indicando las dimensiones a seleccionar.
Dimensiones posibles: Time, bottom_top, south_north, west_east, 
                      south_north_stag, west_east_stag, soil_layers_stag, bottom_top_stag
'''
# Parametros (se obtienen los datos de la variable en el tiempo t y la altura h)
t = 13 # que hora es?
h = 14 # que presion es?
# Seleccion y conversion a matriz numpy
PH_sub = DS.PH.sel(Time = t, bottom_top_stag = h )
PH_numpy = PH_sub.values
print PH_numpy.shape
PHB_sub = DS.PHB.sel(Time = t, bottom_top_stag = h )
PHB_numpy = PHB_sub.values
print PHB_numpy.shape

z0 = (PH_numpy + PHB_numpy) / 9.81

t = 14
PH_sub = DS.PH.sel(Time = t, bottom_top_stag = h )
PH_numpy = PH_sub.values
PHB_sub = DS.PHB.sel(Time = t, bottom_top_stag = h )
PHB_numpy = PHB_sub.values

z1 = (PH_numpy + PHB_numpy) / 9.81

print z0.shape
print z1.shape
z0_ex = np.expand_dims(z0, axis = 0)
z1_ex = np.expand_dims(z1, axis = 0)
print z0_ex.shape
print z1_ex.shape
z0_ex = np.concatenate((z0_ex, z1_ex), axis=0)
print(z0_ex.shape)#
# Guardar en disco la matriz numpy
#np.save('PH_0_0.npy',da_numpy)

In [ ]:
'''
Seleccionar un subset de datos de una variable especifica, indicando las dimensiones a seleccionar.
Dimensiones posibles: Time, bottom_top, south_north, west_east, 
                      south_north_stag, west_east_stag, soil_layers_stag, bottom_top_stag
'''
for h in range (0,34): #Altura con la que se va a crear la matriz
    # Inicializacion con el tiempo 0
    PH_sub = DS.PH.sel(Time = 0, bottom_top_stag = h )
    PH_numpy = PH_sub.values
    print PH_numpy.shape
    PHB_sub = DS.PHB.sel(Time = 0, bottom_top_stag = h )
    PHB_numpy = PHB_sub.values
    print PHB_numpy.shape
    z = (PH_numpy + PHB_numpy) / 9.81
    z_ex = np.expand_dims(z, axis = 0)
    print z_ex.shape

    #Completo con el resto de los tiempos
    for t in range(1, 34):
        PH_sub = DS.PH.sel(Time = t, bottom_top_stag = h )
        PH_numpy = PH_sub.values
        #print PH_numpy.shape
        PHB_sub = DS.PHB.sel(Time = t, bottom_top_stag = h )
        PHB_numpy = PHB_sub.values
        #print PHB_numpy.shape
        zaux = (PH_numpy + PHB_numpy) / 9.81
        zaux_ex = np.expand_dims(zaux, axis = 0)
        z_ex = np.concatenate((z_ex, zaux_ex), axis=0)
    print(z_ex.shape)#
    print z_ex[0]
    #Guardar en disco la matriz numpy
    np.save('z_altura{}_prueba.npy'.format(h),z_ex)

In [ ]:
'''
Listar los atributos de una variable
'''
ds.variables['PH'].__dict__

In [ ]:
'''
Listar las dimensiones de una variable
'''
ds.variables['PHB'].dimensions

In [ ]:
'''
Ver el arreglo con los valores
'''
ds.variables['PHB'][:]

#### 3.2 interplevel

Esta función interpola de manera simple y rápida a una altura o nivel de presión específico. Usa  la interpolación lineal, para interpolar con más opciones se puede usar la función ** vinterp **

*Ejemplo:*

In [ ]:
# Se obtiene la altura en km, al desactivar la opción msl (Median Sea Level) se calcula AGL (Above Ground Level)
#z = wrf.getvar(ds, 'z', timeidx=wrf.ALL_TIMES, msl=False, units='km')
#print z.shape
# Interpolación de la perturbacion geopotencial a 10 km sobre la superficie
#PH = wrf.getvar(ds, 'PH', timeidx=wrf.ALL_TIMES)
#print PH.shape
#PH_10 = wrf.interplevel(PH,z,10)

print 